In [1]:
from pytrips.ontology import get_ontology as tripsont
from pytrips.structures import TripsType
from nltk.corpus.reader.wordnet import Synset
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')
from collections import defaultdict
from collections import namedtuple
import random

import math
from scipy.stats import spearmanr, pearsonr

nouns = list(wn.all_synsets(pos=wn.NOUN))
num = 100
noun_sample = random.sample(nouns, num)

In [2]:
"""
Default node weights, can be overrided for variations
"""
_node_weights = defaultdict(lambda: 1)

_node_weights["fakeroot"] = 0

_dcache = {}


def _fallback(x, func, nonzero=False, fbs=-1):
    """
    Basic null checks for selecting a value from a list
    """
    if (len(x) == 0) or (sum(x) == 0 and nonzero):
        return fbs
    return func(x)

"""
Pick a value from a list
"""
ListStrategy = {
    'choose': lambda f: lambda b: lambda x: _fallback(x, f, fbs=-1),
    'min': lambda x: _fallback(x, min),
    'bmin': lambda b: lambda x: _fallback(x, min, fbs=b),
    'max': lambda x: _fallback(x, max),
    'bmax': lambda b: lambda x: _fallback(x, max, fbs=b),
    'average': lambda x: _fallback(x, lambda l: sum(l)/len(l), nonzero=True)
}


"""
Find the last common element between two lists
"""
def last_overlap(v1, v2, aligned=None):
    if not v1 or not v2:
        return aligned
    elif v1[0] != v2[0]:
        return aligned
    else:
        return last_overlap(v1[1:], v2[1:], aligned=v1[0])

class SemNode:
    def __init__(self, node):
        self._node = node
    
    def __eq__(self, other):
        if issubclass(type(other), SemNode):
            if type(other.content) == type(self.content):
                return other.content == self.content
        return False
    
    def __hash__(self):
        return hash(self.__repr__())
    
    def __repr__(self):
        return "<SemNode: {}>".format(self.content.__repr__())
    
    def __str__(self):
        return "<SemNode: {}>".format(self.name)
    
    @property
    def name(self):
        """
        return canonical name for node
        """
        return str(self.content)
    
    @property
    def adjacent(self, label=None):
        """
        Return connected elements by label.  Useful for following non-hypernym relations in wordnet
        Can be used to implement other connections.
        Override as necessary.
        """
        return []
    
    @property
    def root(self):
        """
        Check if the node is a root of some sort.  Override as necessary for different resources.
        """
        return not self.parents
    
    @property
    def parents(self):
        """
        Get all parents for a node.  Perform any cross-resource cutoffs.
        """
        pass
    
    @property
    def children(self):
        """
        Get all children for a node.  Perform any cross resource cutoffs.
        """
        pass
    
    @property
    def content(self):
        """
        Return the wrapped node.
        """
        return self._node
    
    @property
    def resource(self):
        """
        Return the name of the relevant resource
        """
        return "default"
    
    def weight(self, weights=None):
        """
        Get resource-based weight of node.  Pass a weight dictionary as necessary.
        TODO: add an argument to pass individual weights
        """
        if not weights:
            weights = _node_weights
        return weights[self.resource]
    
    @staticmethod
    def path_depth(path, weights=None):
        """
        Get the total depth of a path, defined as a list of nodes.
        Does not check validity of path
        """
    
        return sum([p.weight(weights=weights) for p in path])
    
    @staticmethod
    def depth(node, weights=None, strategy='max'):
        """
        Get the depth of a node to a root using a weight dictionary and selection strategy.
        Default is minimum depth from any root.
        """
        if strategy in _dcache:
            if node.content in _dcache:
                return _dcache[strategy][node.content]
        else:
            _dcache[strategy] = {}
        weighted = [SemNode.path_depth(p, weights=weights) for p in node.paths_to_root()]
        _dcache[strategy][node.content] = ListStrategy[strategy](weighted) - 1
        return _dcache[strategy][node.content]
    
    @staticmethod
    def make(node):
        """
        Make a node based on the input type.
        Should add parameter dictionary to pass on to children
        """
        if type(node) is TripsType:
            return TripsNode(node)
        elif type(node) is Synset:
            return WordNetNode(node)
        elif type(node) is str:
            return WordNode(node)
        elif type(node) is SemNode:
            return node
        else:
            return None
    
    def paths_to_root(self):
        """
        Find all paths to a root based on hierarchy rules.
        Some resources return only one (Trips), others may return multiple (WordNet)
        """
        if self.root:
            return [[self]]
        res = []
        for c in self.parents:
            ptrs = c.paths_to_root()
            res.extend([t + [self] for t in ptrs if self not in t])
        return res
    
    def lcs_set(self, other):
        """
        Find the set of Lowest Common Subsumers for a node.  
        Some resources have only one (Trips) other can have multiple (WordNet)
        """
        lcs = [last_overlap(p,q) for p in self.paths_to_root() for q in other.paths_to_root()]
        filtered = [x for x in lcs if x]
        if not filtered:
            return [TripsNode(tripsont()["root"])]
        return filtered
    
    def wupalmer(self, other, weights=None, depth_strategy='min', lcs_strategy='max'):
        """
        return cross-wupalmer measure using provided weights, depth_strategy and lcs_strategy
        depth_strategy: Choose max, min, or average depth over all paths
        lcs_strategy: Choose max, min, or average depth of lcs over all alternatives
        """
        if not issubclass(type(other), SemNode):
            other = SemNode.make(other) # this would break passing in an arbitrary maker object
        lcs_depth = ListStrategy[lcs_strategy]([SemNode.depth(d, weights, depth_strategy) for d in self.lcs_set(other)])
        sd = SemNode.depth(self, weights, depth_strategy)
        od = SemNode.depth(other, weights, depth_strategy)
        # nlwup = self.content.wup_similarity(other.content, simulate_root=True)
        return 2*(lcs_depth)/(sd + od)
    
    def path_similarity(self, other, weights=None, depth_strategy='max', lcs_strategy='max'):
        """
        Like wupalmer, except (d(s1) + d(s2) - 2 * lcs(s1,s2))
        """
        if not issubclass(type(other), SemNode):
            other = SemNode.make(other) # this would break passing in an arbitrary maker object
        lcs_depth = ListStrategy[lcs_strategy]([SemNode.depth(d, weights, depth_strategy) for d in self.lcs_set(other)])
        sd = SemNode.depth(self, weights, depth_strategy) + self.weight(weights=weights)
        od = SemNode.depth(other, weights, depth_strategy) + other.weight(weights=weights)
        return (sd + od) - 2 * lcs_depth

In [3]:
class WordNode(SemNode):
    """
    Take a "word.pos" element as a node in the generalized hierarchy.
    """
    def resource(self):
        return "word"
    
    @property
    def name(self):
        return self.content
    
    @property
    def children(self):
        return []
    
    def word_pos(self):
        if "." in self._node:
            return self._node.split(".")
        return self._node, None
    
    @property
    def parents(self):
        """
        Lookup all TripsTypes, lookup all Wordnet Types.
        """
        w, p = self.word_pos()
        wordnet = wn.synsets(w, p)
        trips = tripsont().get_word(w, p)
        return [SemNode(c) for c in wordnet+trips]

class FakeRoot(SemNode):
    """
    FakeRoot for completeness purposes
    """
    def __init__(self):
        super(FakeRoot, self).__init__("fakeroot")
        
    def resource(self):
        return "fakeroot"
    
    @property
    def name(self):
        return "fakeroot"
    
    @property
    def parents(self):
        return []
    
    @property
    def children(self):
        return []
        

class TripsNode(SemNode):
    @property
    def name(self):
        return self.content.name
    
    @property
    def parents(self):
        return [SemNode.make(self._node.parent)]

    @property
    def children(self):
        return [SemNode.make(c) for c in self._node.children] + [SemNode.make(c) for c in self._node.wordnet]
    
    @property
    def resource(self):
        return "trips"
    
    @property
    def root(self):
        return self._node.depth == 0

    
class WordNetNode(SemNode):
    @property
    def name(self):
        return self.content.name()
    
    @property
    def parents(self):  
        # NOTE: actually this is a little bit of a problem because we're not taking
        #       WN hypernyms
        tt = tripsont()[self._node]# + self._node.hypernyms()
        if not tt:
            tt = self._node.hypernyms()
        if not tt:
            return [FakeRoot()]
        return [SemNode.make(p) for p in tt]

    @property
    def children(self):
        return [SemNode.make(c) for c in self._node.hyponyms()]
    
    @property
    def resource(self):
        return "wordnet"
    

# tests

In [4]:
# equality

cat1 = wn.synset("cat.v.1")
cat2 = wn.synset("cat.v.1")

wcat1 = WordNetNode(cat1)
wcat2 = WordNetNode(cat2)

assert issubclass(type(wcat1), SemNode)
assert wcat1 == wcat2

# hypernyms

animal = SemNode.make(tripsont()["nonhuman-animal"])
mammal = SemNode.make(tripsont()["mammal"])
assert [mammal] == animal.parents


abbess = SemNode.make(wn.synset("abbess.n.1"))
scand  = SemNode.make(wn.synset("scandinavia.n.2"))
print(abbess.paths_to_root())
print(scand.paths_to_root())

wn.synset("scandinavia.n.2").hypernyms()

2019-05-08 01:08:43,125 : pytrips : INFO : Loading ontology
2019-05-08 01:08:43,127 : root : INFO : Getting ontology.json - version 0.1.15
2019-05-08 01:08:43,431 : pytrips : INFO : Loaded ontology
2019-05-08 01:08:43,431 : pytrips : INFO : Loading lexicon
2019-05-08 01:08:43,432 : root : INFO : Getting words.json - version 0.1.15
2019-05-08 01:08:43,597 : pytrips : INFO : Loaded lexicon
2019-05-08 01:08:43,598 : pytrips : INFO : RETURN24549 has 3 senses
2019-05-08 01:08:43,599 : pytrips : INFO : RETURNS24551 has 3 senses
2019-05-08 01:08:43,600 : pytrips : INFO : DAY7612 has 2 senses
2019-05-08 01:08:43,600 : pytrips : INFO : DISARM8671 has 2 senses
2019-05-08 01:08:43,601 : pytrips : INFO : SUPPLEMENT28833 has 2 senses
2019-05-08 01:08:43,602 : pytrips : INFO : SUPPLEMENTS28835 has 2 senses
2019-05-08 01:08:43,603 : pytrips : INFO : DITHER9083 has 2 senses
2019-05-08 01:08:43,604 : pytrips : INFO : HIT14034 has 3 senses
2019-05-08 01:08:43,605 : pytrips : INFO : STUMP28527 has 2 sens

2019-05-08 01:08:43,719 : pytrips : INFO : CARD4458 has 2 senses
2019-05-08 01:08:43,720 : pytrips : INFO : IMPAIRMENTS14663 has 2 senses
2019-05-08 01:08:43,720 : pytrips : INFO : CERTIFY4850 has 2 senses
2019-05-08 01:08:43,721 : pytrips : INFO : INTERACTION15460 has 5 senses
2019-05-08 01:08:43,722 : pytrips : INFO : ORIGINATE20107 has 2 senses
2019-05-08 01:08:43,723 : pytrips : INFO : SPRINKLE27830 has 2 senses
2019-05-08 01:08:43,723 : pytrips : INFO : SPRINKLE27822 has 2 senses
2019-05-08 01:08:43,724 : pytrips : INFO : TEST29709 has 2 senses
2019-05-08 01:08:43,725 : pytrips : INFO : FIGURE11437 has 2 senses
2019-05-08 01:08:43,726 : pytrips : INFO : ENTHUSIASM10272 has 2 senses
2019-05-08 01:08:43,727 : pytrips : INFO : COME5911 has 3 senses
2019-05-08 01:08:43,727 : pytrips : INFO : MERGE18108 has 3 senses
2019-05-08 01:08:43,728 : pytrips : INFO : AS-OPPOSED-TO1916 has 2 senses
2019-05-08 01:08:43,728 : pytrips : INFO : AS-WELL-AS1914 has 2 senses
2019-05-08 01:08:43,729 : p

2019-05-08 01:08:43,804 : pytrips : INFO : FLIGHT11726 has 2 senses
2019-05-08 01:08:43,805 : pytrips : INFO : TERMINATE29644 has 2 senses
2019-05-08 01:08:43,806 : pytrips : INFO : DIFFER8463 has 2 senses
2019-05-08 01:08:43,806 : pytrips : INFO : REQUEST24306 has 3 senses
2019-05-08 01:08:43,807 : pytrips : INFO : REQUEST24298 has 3 senses
2019-05-08 01:08:43,807 : pytrips : INFO : RAP23384 has 2 senses
2019-05-08 01:08:43,807 : pytrips : INFO : COOL6763 has 2 senses
2019-05-08 01:08:43,808 : pytrips : INFO : COOL6758 has 2 senses
2019-05-08 01:08:43,809 : pytrips : INFO : PROHIBITION22628 has 2 senses
2019-05-08 01:08:43,810 : pytrips : INFO : WITHER33069 has 2 senses
2019-05-08 01:08:43,812 : pytrips : INFO : ALOT1170 has 2 senses
2019-05-08 01:08:43,812 : pytrips : INFO : ALOT1168 has 2 senses
2019-05-08 01:08:43,813 : pytrips : INFO : APPEAL1592 has 2 senses
2019-05-08 01:08:43,814 : pytrips : INFO : SUPPER28830 has 2 senses
2019-05-08 01:08:43,814 : pytrips : INFO : NEARLY19090 

2019-05-08 01:08:43,891 : pytrips : INFO : WIDEN32915 has 2 senses
2019-05-08 01:08:43,892 : pytrips : INFO : SORTA27378 has 3 senses
2019-05-08 01:08:43,893 : pytrips : INFO : COMPANY6025 has 2 senses
2019-05-08 01:08:43,893 : pytrips : INFO : COMPLAIN6092 has 3 senses
2019-05-08 01:08:43,894 : pytrips : INFO : ISOLATE15796 has 2 senses
2019-05-08 01:08:43,895 : pytrips : INFO : HINDER13999 has 2 senses
2019-05-08 01:08:43,896 : pytrips : INFO : ABSORB374 has 2 senses
2019-05-08 01:08:43,896 : pytrips : INFO : CORRODE6869 has 2 senses
2019-05-08 01:08:43,897 : pytrips : INFO : THEORETICAL29816 has 2 senses
2019-05-08 01:08:43,899 : pytrips : INFO : USE31858 has 2 senses
2019-05-08 01:08:43,899 : pytrips : INFO : USE31850 has 2 senses
2019-05-08 01:08:43,900 : pytrips : INFO : DREAD9381 has 2 senses
2019-05-08 01:08:43,901 : pytrips : INFO : ORIGINATE20107 has 2 senses
2019-05-08 01:08:43,901 : pytrips : INFO : DIGEST8496 has 2 senses
2019-05-08 01:08:43,902 : pytrips : INFO : CONJECTU

2019-05-08 01:08:43,983 : pytrips : INFO : VERIFY32090 has 2 senses
2019-05-08 01:08:43,984 : pytrips : INFO : TOUCH30418 has 4 senses
2019-05-08 01:08:43,985 : pytrips : INFO : TOUGH30429 has 2 senses
2019-05-08 01:08:43,985 : pytrips : INFO : BUILD3946 has 2 senses
2019-05-08 01:08:43,986 : pytrips : INFO : REPUDIATIONS24273 has 2 senses
2019-05-08 01:08:43,987 : pytrips : INFO : FRESH12223 has 2 senses
2019-05-08 01:08:43,988 : pytrips : INFO : DELETIONS7944 has 2 senses
2019-05-08 01:08:43,988 : pytrips : INFO : CHECK-OUT5045 has 2 senses
2019-05-08 01:08:43,989 : pytrips : INFO : CHECK-IN5036 has 3 senses
2019-05-08 01:08:43,990 : pytrips : INFO : CHECK5031 has 6 senses
2019-05-08 01:08:43,990 : pytrips : INFO : CHECK5025 has 2 senses
2019-05-08 01:08:43,991 : pytrips : INFO : CONVERSE6713 has 2 senses
2019-05-08 01:08:43,992 : pytrips : INFO : LET16841 has 2 senses
2019-05-08 01:08:43,993 : pytrips : INFO : GIVE12758 has 3 senses
2019-05-08 01:08:43,994 : pytrips : INFO : GIVE127

...

2019-05-08 01:08:47,178 : pytrips : INFO : CUP7344 has 2 senses
2019-05-08 01:08:47,179 : pytrips : INFO : LENGTHEN16789 has 2 senses
2019-05-08 01:08:47,179 : pytrips : INFO : LENGTHEN16784 has 2 senses
2019-05-08 01:08:47,180 : pytrips : INFO : MARK17732 has 2 senses
2019-05-08 01:08:47,181 : pytrips : INFO : INTERPRET15528 has 2 senses
2019-05-08 01:08:47,181 : pytrips : INFO : FAKE11117 has 2 senses
2019-05-08 01:08:47,182 : pytrips : INFO : DIVEST9096 has 2 senses
2019-05-08 01:08:47,183 : pytrips : INFO : ENERGIZE10086 has 2 senses
2019-05-08 01:08:47,184 : pytrips : INFO : LOOSEN17322 has 2 senses
2019-05-08 01:08:47,185 : pytrips : INFO : NOTICE19475 has 3 senses
2019-05-08 01:08:47,185 : pytrips : INFO : BLISTER3264 has 2 senses
2019-05-08 01:08:47,186 : pytrips : INFO : SUBSTITUTE28642 has 2 senses
2019-05-08 01:08:47,187 : pytrips : INFO : SHELLAC26140 has 2 senses
2019-05-08 01:08:47,187 : pytrips : INFO : PREFERABLY22286 has 2 senses
2019-05-08 01:08:47,188 : pytrips : INF

2019-05-08 01:08:47,270 : pytrips : INFO : ENTRUST10297 has 2 senses
2019-05-08 01:08:47,270 : pytrips : INFO : JAM15891 has 2 senses
2019-05-08 01:08:47,271 : pytrips : INFO : DISTRIBUTE9058 has 2 senses
2019-05-08 01:08:47,272 : pytrips : INFO : CROWN7254 has 2 senses
2019-05-08 01:08:47,272 : pytrips : INFO : ABSORPTIONS376 has 2 senses
2019-05-08 01:08:47,273 : pytrips : INFO : CHIDE5152 has 2 senses
2019-05-08 01:08:47,274 : pytrips : INFO : FACILITY11065 has 2 senses
2019-05-08 01:08:47,274 : pytrips : INFO : FATTEN11219 has 3 senses
2019-05-08 01:08:47,275 : pytrips : INFO : KILL16245 has 3 senses
2019-05-08 01:08:47,276 : pytrips : INFO : IMPROVE14749 has 2 senses
2019-05-08 01:08:47,277 : pytrips : INFO : SMOKE26978 has 2 senses
2019-05-08 01:08:47,278 : pytrips : INFO : IGNITE14602 has 2 senses
2019-05-08 01:08:47,279 : pytrips : INFO : LODGE17229 has 2 senses
2019-05-08 01:08:47,279 : pytrips : INFO : COERCE5755 has 2 senses
2019-05-08 01:08:47,280 : pytrips : INFO : RESPOND

2019-05-08 01:08:52,685 : pytrips.helpers : INFO : no synset found for another%3:00:00::


[[<SemNode: ont::root>, <SemNode: ont::any-sem>, <SemNode: ont::referential-sem>, <SemNode: ont::phys-object>, <SemNode: ont::natural-object>, <SemNode: ont::organism>, <SemNode: ont::animal>, <SemNode: ont::vertebrate>, <SemNode: ont::mammal>, <SemNode: ont::person>, <SemNode: Synset('abbess.n.01')>]]
[[<SemNode: 'fakeroot'>, <SemNode: Synset('scandinavia.n.02')>]]


[]

# Evaluation tools

from collections import namedtuple, Counter
from tqdm import tnrange, tqdm_notebook

Comp = namedtuple("comp", ["n1", "n2", "normal", "cross"])
def comp_string(c):
    return """
    node1: {}
    node1: {}
    wordnet only wup: {}
    cross wupalmer:   {}
    """.format(c.n1, c.n2, c.normal, c.cross)

def compare_wup(n1, n2):
    cross = SemNode.make(n1).wupalmer(SemNode.make(n2))
    normal= n1.wup_similarity(n2)
    return Comp(n1, n2, normal, cross)

res = []
for i in tqdm_notebook(range(num), desc="i"):
    for j in range(i, num):
        res.append(compare_wup(noun_sample[i], noun_sample[j]))

# count data
ranges_normal = Counter()
ranges_cross = Counter()
for i in res:
    ranges_normal[i.normal] += 1
    ranges_cross[i.cross] += 1

def binning(data, n=10):
    bin_width = 1/n
    binned = []
    for d in data:
        try:
            binned.append(int(n*d)/n)
        except:
            print(n, d, int(n*d))
    return binned

def bin_count(data, n=10):
    b_counter = Counter()
    bin_width = 1/n
    binned = []
    for d in data:
        try:
            b_counter[int(n*d)/n] += 1
        except:
            print(n, d, int(n*d))
    return b_counter

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

bins=25
data = binning([i.normal for i in res], bins)

counted = sorted(bin_count([i.normal for i in res], n=bins).items(), key=lambda x: x[0])
for c in counted:
    print(c)


plt.hist(data, density=True, bins=bins)
plt.ylabel('WordnetOnly');

data = binning([i.cross for i in res if i.cross <= 1], bins)

counted = sorted(bin_count([i.cross for i in res], n=bins).items(), key=lambda x: x[0])
for c in counted:
    print(c)

for i in res:
    if 0 > i.cross or 1 < i.cross:
        print(i)

plt.hist(data, density=True, bins=bins)
plt.ylabel('Trips-WUPalmer');

In [5]:
import pandas as pd

simlex = pd.read_csv("../SimLex-999/SimLex-999.txt", sep="\t")

In [30]:
node_weights = defaultdict(lambda: 1)

def pops(l):
    if l:
        return l[0]
    return None

def collects(l):
    return [y for y in l if y]

def n_or_v2(synset):
    return synset.pos() in [wn.NOUN, wn.VERB]

def n_or_v(synset):
    return synset.pos() in [wn.NOUN]

def get_pos_men(x):
    if x in "nv":
        return x
    elif x == "j":
        return "ars"
    else:
        return "nvars"

def hybrid_wup(s1, s2):
    if n_or_v2(s1.content) and n_or_v2(s2.content):
        return s1.content.wup_similarity(s2.content)
    return s1.wupalmer(s2)

def resnik(s1, s2, ic_corpus):
    if n_or_v(s1) and n_or_v(s2):
        return s1.res_similarity(s2, ic_corpus)
    return -1 #s1.wup_similarity(s2)

def _list_fallback(func, fallback_func, args, both=False):
    result = func(*args)
    if not result or both:
        result += fallback_func(*args)
    return result

sim_strategy = {
    "mfs": lambda x, p: [SemNode.make(pops(wn.synsets(x, p)))],
    "average": lambda x, p: [SemNode.make(s) for s in wn.synsets(x, p)],
    "word": lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)],
    "lookup": lambda x, p: _list_fallback(
        lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)], 
        lambda x, p: [SemNode.make(pops(wn.synsets(x, p)))],
        [x, p]
    ),
    "lookupall": lambda x, p: _list_fallback(
        lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)], 
        lambda x, p: [SemNode.make(pops(wn.synsets(x, p)))],
        [x, p], both=True
    ),
    "both": lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)] + [SemNode.make(pops(wn.synsets(x, p)))]
}

sim_metric = {
    "cross" : lambda x, y: x.wupalmer(y, node_weights),
    "tripspath" : lambda x, y: x.path_similarity(y, weights=node_weights),
    "normal": lambda x, y: x.content.wup_similarity(y.content),
    "resnik_brown": lambda x, y: resnik(x.content, y.content, brown_ic),
    "resnik_semcor": lambda x, y: resnik(x.content, y.content, semcor_ic),
    "hybrid": hybrid_wup
}

def similarity_test(word1, word2, pos=None, metric="cross", strategy="average", select1="max", select2="max", cheats=-1):
    #return cheats
    if not pos:
        pos = "nvar"
    elif type(pos) is tuple:
        pos = "nvar"
    else:
        pos = pos.lower()[0]
    metric = sim_metric[metric]
    strategy = sim_strategy[strategy]
    results = []
    for x in pos:
        word1_node = collects(strategy(word1, x))
        word2_node = collects(strategy(word2, x))
        if word1_node and word2_node:
            scores = collects([metric(x, y) for x in word1_node for y in word2_node])
            results.append(ListStrategy[select1](scores))
    if results:
        #print("{}\t{} -> ({}) | {}".format(word1, word2, ListStrategy[select2](results), results))
        return ListStrategy[select2](results)
    return -1 # fallback of 0.5

def similarity_test2(word1, word2, pos, metric="cross", strategy="average", select1="max", select2="max", cheats=-1):
    if not pos:
        pos = ("nvars", "nvars")
    pos1, pos2 = pos
    metric = sim_metric[metric]
    strategy = sim_strategy[strategy]
    results = []
    for x in pos1:
        for y in pos2:
            word1_node = collects(strategy(word1, x))
            word2_node = collects(strategy(word2, y))
            if word1_node and word2_node:
                scores = collects([metric(x, y) for x in word1_node for y in word2_node])
                results.append(ListStrategy[select1](scores))
    if results:
        #print("{}\t{} -> ({}) | {}".format(word1, word2, ListStrategy[select2](results), results))
        return ListStrategy[select2](results)
    return -1 # fallback of 0.5
    

In [7]:

def get_valid_scores(l1, l2):
    res1 = []
    res2 = []
    for x, y in zip(l1, l2):
        if y >= 0:
            res1.append(x)
            res2.append(y)
    print("dropping:", len(l1) - len(res1))
    return res1, res2

def confidence(rho, n):
    import math
    stderr = 1.0 / math.sqrt(n - 3)
    delta = 1.96 * stderr
    #print(rho, n, delta)
    lower = math.tanh(math.atanh(rho) - delta)
    upper = math.tanh(math.atanh(rho) + delta)
    return (lower, upper)

In [43]:
SimExperiment = namedtuple("SimExperiment", 
                           ["name", "data", "metric", "strategy", "select1", "select2", "pivot"]
                          )
SimTask = namedtuple("SimTask", ["word1", "word2", "gold", "pos"])
SimTaskResults = namedtuple("SimTaskResults", ["experiment", "instances", "spearman","pearson"])

def ws353(name, metric, strategy, select1="max", select2="max", fname="combined"):
    wordsim = pd.read_csv("../wordsim353/{}.tab".format(fname), sep="\t")
    res = []
    for i, row in wordsim.iterrows():
        res.append(SimTask(row[0], row[1], row[2], None))
    print(len(res))
    pivot=0
    if fname == "relatedness":
        pivot=0
    return SimExperiment(name, res, metric, strategy, select1, select2, pivot)

def slex(name, metric, strategy, select1="max", select2="max", fname="nvars"):
    wordsim = pd.read_csv("../SimLex-999/SimLex-999.txt".format(fname), sep="\t")
    res = []
    for i, row in wordsim.iterrows():
        if row[2].lower() in fname:
            res.append(SimTask(row[0], row[1], row[3], row[2].lower()))
    print(len(res))
    pivot=0
    if fname == "relatedness":
        pivot=0
    return SimExperiment(name, res, metric, strategy, select1, select2, pivot)

def MEN(name, metric, strategy, select1="max", select2="max", fname="dev"):
    wordsim = pd.read_csv("../MEN/MEN_dataset_lemma_form.{}".format(fname), sep=" ")
    res = []
    for i, row in wordsim.iterrows():
        w1, w2, score = row[0][:-2], row[1][:-2], row[2], 
        pos1, pos2 = get_pos_men(row[0][-1]), get_pos_men(row[1][-1])
        if pos1.lower() not in "n" and pos2.lower() not in "n":
            res.append(SimTask(w1, w2, score, (pos1, pos2)))
    print(len(res))
    pivot=0
    if fname == "relatedness":
        pivot=0
    return SimExperiment(name, res, metric, strategy, select1, select2, pivot)

def MENNatural(name, metric, strategy, select1="max", select2="max", fname="dev"):
    wordsim = pd.read_csv("../MEN/MEN_dataset_natural_form_full", sep="\t")
    res = []
    for i, row in wordsim.iterrows():
        w1, w2, score = row[0], row[1], row[2], 
        res.append(SimTask(w1, w2, score, None))
    print(len(res))
    pivot=0
    if fname == "relatedness":
        pivot=0
    return SimExperiment(name, res, metric, strategy, select1, select2, pivot)

def MENIndiv(name, metric, strategy, select1="max", select2="max", fname="elias"):
    wordsim = pd.read_csv("../MEN/agreement/{}-men-ratings.txt".format(fname), sep=" ")
    res = []
    for i, row in wordsim.iterrows():
        w1, w2, score = row[0], row[1], row[2], 
        res.append(SimTask(w1, w2, score, None))
    print(len(res))
    pivot=0
    if fname == "relatedness":
        pivot=0
    return SimExperiment(name, res, metric, strategy, select1, select2, pivot)

def run_experiment(exp, postuple=False):
    results = []
    st = similarity_test
    if postuple:
        st = similarity_test2
    for d in exp.data:
        r = abs(exp.pivot - st(d.word1, d.word2, pos=d.pos, 
                metric=exp.metric, 
                strategy=exp.strategy, 
                select1=exp.select1, select2=exp.select2,
                cheats=d.gold
               )) * 1/(1 - exp.pivot)
        g = d.gold
        if r >= 0:
            results.append((r,g))
    x = [r for r, g in results]
    y = [g for r, g in results]
    return SimTaskResults(exp, x, spearmanr(x, y), None)#pearsonr(x, y))

def experiment_string(exp, pandas=True, dataframe=None):
    columns = "name metric candidate choice1 choice2 instances spearmanr conf_low conf_high".split()
    if dataframe is None:
        dataframe = pd.DataFrame(columns=columns)
    e = exp.experiment
    if pandas:
        clow, chigh = confidence(exp.spearman.correlation, len(exp.instances))
        values = [e.name, e.metric, e.strategy, e.select1, e.select2, len(exp.instances),
        exp.spearman.correlation, clow, chigh] #exp.spearman.pvalue, 
        #exp.pearson[0], exp.pearson[1]]
        res = pd.DataFrame({v: [r] for r, v in zip(values, columns)})
        return dataframe.append(res)
    return """
    ---
    name:      {}
    metric:    {}
    candidate: {}
    choice:    {}/{}
    --- 
    instances:    {}
    spearman rho: {}
    p-value:      {}
    
    pearson rho:  {}
    p-value:      {}
    ================
    """.format(
        e.name, e.metric, e.strategy, e.select1, e.select2, len(exp.instances),
        exp.spearman.correlation, exp.spearman.pvalue, 
        exp.pearson[0], exp.pearson[1]
    )
    

In [9]:
# Tripswordnet
df = None
_dcache={}
fname = ["n", "v", "nv", "a"][0]

test = [ws353, slex][1]

node_weights["fakeroot"] = 1
node_weights["wordnet"] = 1
node_weights["trips"] = 1
node_weights["word"] = 1

df = experiment_string(run_experiment(test("base", "normal", strategy="average", select1="max", select2="max", fname=fname)))

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="average", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("base", "normal", strategy="mfs", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="mfs", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="lookup", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="both", select1="max", select2="max", fname=fname)), dataframe=df)



df

666


/Users/dev/.pyenv/versions/3.5.5/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


666
666
666
666
666


,candidate,choice1,choice2,conf_high,conf_low,instances,metric,name,spearmanr
0,average,max,max,0.601005,0.494863,666,normal,base,0.550152
0,average,max,max,0.455143,0.326519,666,cross,Trips-Wordnet,0.392750
0,mfs,max,max,0.533943,0.416462,666,normal,base,0.477333
0,mfs,max,max,0.445339,0.315490,666,cross,Trips-Wordnet,0.382300
0,lookup,max,max,0.327969,0.186116,666,cross,Trips-Wordnet,0.258435
0,both,max,max,0.413981,0.280445,666,cross,Trips-Wordnet,0.348983


In [10]:
# Tripswordnet
df = None
_dcache={}
fname = ["n", "v", "nv", "vars", "a", "nvars"][3]

test = [ws353, slex][1]

node_weights["fakeroot"] = 1
node_weights["wordnet"] = 1
node_weights["trips"] = 1
node_weights["word"] = 1

df = experiment_string(run_experiment(test("base", "normal", strategy="average", select1="max", select2="max", fname=fname)))

df = experiment_string(run_experiment(test("Trips-Wordnet", "hybrid", strategy="average", select1="max", select2="max", fname=fname)), dataframe=df)

#df = experiment_string(run_experiment(test("base", "normal", strategy="mfs", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-av", "cross", strategy="average", select1="max", select2="max", fname=fname)), dataframe=df)

#df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="lookup", select1="max", select2="max", fname=fname)), dataframe=df)

#df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="both", select1="max", select2="max", fname=fname)), dataframe=df)
fname = ["n", "v", "nv", "vars", "a", "nvars"][1]
df = experiment_string(run_experiment(test("Trips-Wordnet-v", "cross", strategy="average", select1="max", select2="max", fname=fname)), dataframe=df)



df

333
333
333
222


,candidate,choice1,choice2,conf_high,conf_low,instances,metric,name,spearmanr
0,average,max,max,0.276100,0.067563,333,normal,base,0.173779
0,average,max,max,0.428051,0.237119,333,hybrid,Trips-Wordnet,0.336033
0,average,max,max,0.561718,0.396552,333,cross,Trips-Wordnet-av,0.483426
0,average,max,max,0.591637,0.392957,222,cross,Trips-Wordnet-v,0.498822


In [44]:
# Tripswordnet
df = None
#_dcache={}
fname = ["test", "dev"][1]

test = [ws353, slex, MEN][2]

node_weights["fakeroot"] = 1
node_weights["wordnet"] = 1
node_weights["trips"] = 1
node_weights["word"] = 1

postuple = True

df = experiment_string(run_experiment(test("base", "normal", strategy="average", select1="max", select2="max", fname=fname), postuple=postuple))

#df = experiment_string(run_experiment(test("Trips-Wordnet", "hybrid", strategy="average", select1="max", select2="max", fname=fname), postuple=True), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-pt", "cross", strategy="average", select1="max", select2="max", fname=fname), postuple=postuple), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-npt", "cross", strategy="both", select1="max", select2="max", fname=fname), postuple=postuple), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-npt", "cross", strategy="lookup", select1="max", select2="max", fname=fname), postuple=postuple), dataframe=df)


df

110
110
110
110


,candidate,choice1,choice2,conf_high,conf_low,instances,metric,name,spearmanr
0,average,max,max,0.538847,0.219916,110,normal,base,0.391058
0,average,max,max,0.284510,-0.086164,110,cross,Trips-Wordnet-pt,0.102739
0,both,max,max,0.194398,-0.180072,110,cross,Trips-Wordnet-npt,0.007423
0,lookup,max,max,0.195608,-0.178854,110,cross,Trips-Wordnet-npt,0.008681


In [40]:
# Tripswordnet
df = None
#_dcache={}
fname = ["test", "dev"][0]

test = [ws353, slex, MEN][2]

node_weights["fakeroot"] = 1
node_weights["wordnet"] = 1
node_weights["trips"] = 1
node_weights["word"] = 1

postuple = True

df = experiment_string(run_experiment(test("base", "normal", strategy="average", select1="max", select2="max", fname=fname), postuple=postuple))

#df = experiment_string(run_experiment(test("Trips-Wordnet", "hybrid", strategy="average", select1="max", select2="max", fname=fname), postuple=True), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-pt", "cross", strategy="average", select1="max", select2="max", fname=fname), postuple=postuple), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-npt", "cross", strategy="both", select1="max", select2="max", fname=fname), postuple=postuple), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-npt", "cross", strategy="lookup", select1="max", select2="max", fname=fname), postuple=postuple), dataframe=df)

df

7
7
7
7


,candidate,choice1,choice2,conf_high,conf_low,instances,metric,name,spearmanr
0,average,max,max,0.854944,-0.595247,7,normal,base,0.286039
0,average,max,max,0.913431,-0.390291,7,cross,Trips-Wordnet-pt,0.513783
0,both,max,max,0.810287,-0.681622,7,cross,Trips-Wordnet-npt,0.146795
0,lookup,max,max,0.786429,-0.715322,7,cross,Trips-Wordnet-npt,0.081818


In [15]:
# Tripswordnet
df = None
_dcache={}
fname = ["test", "dev"][0]

test = [ws353, slex, MEN, MENNatural][3]

node_weights["fakeroot"] = 1
node_weights["wordnet"] = 1
node_weights["trips"] = 1
node_weights["word"] = 1

df = experiment_string(run_experiment(test("base", "normal", strategy="average", select1="max", select2="max", fname=fname), postuple=False))

#df = experiment_string(run_experiment(test("Trips-Wordnet", "hybrid", strategy="average", select1="max", select2="max", fname=fname), postuple=True), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-pt", "cross", strategy="average", select1="max", select2="max", fname=fname), postuple=False), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-npt", "cross", strategy="both", select1="max", select2="max", fname=fname), postuple=False), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-npt", "cross", strategy="lookup", select1="max", select2="max", fname=fname), postuple=False), dataframe=df)




df

2999
2999
2999
2999


,candidate,choice1,choice2,conf_high,conf_low,instances,metric,name,spearmanr
0,average,max,max,0.349194,0.284810,2999,normal,base,0.317367
0,average,max,max,0.328396,0.263078,2999,cross,Trips-Wordnet-pt,0.296083
0,both,max,max,0.332911,0.267790,2999,cross,Trips-Wordnet-npt,0.300701
0,lookup,max,max,0.304007,0.237678,2999,cross,Trips-Wordnet-npt,0.271165


In [32]:
# Tripswordnet
df = None
_dcache={}
fname = ["elias", "marcos"][0]

test = MENIndiv
postuple=True

node_weights["fakeroot"] = 1
node_weights["wordnet"] = 1
node_weights["trips"] = 1
node_weights["word"] = 1

df = experiment_string(run_experiment(test("base", "normal", strategy="average", select1="max", select2="max", fname=fname), postuple=postuple))

#df = experiment_string(run_experiment(test("Trips-Wordnet", "hybrid", strategy="average", select1="max", select2="max", fname=fname), postuple=True), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-pt", "cross", strategy="average", select1="max", select2="max", fname=fname), postuple=postuple), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-npt", "cross", strategy="both", select1="max", select2="average", fname=fname), postuple=postuple), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet-npt", "cross", strategy="lookup", select1="max", select2="max", fname=fname), postuple=postuple), dataframe=df)


df



2999
2999
2999
2999


,candidate,choice1,choice2,conf_high,conf_low,instances,metric,name,spearmanr
0,average,max,max,0.342779,0.278100,2999,normal,base,0.310799
0,average,max,max,0.350471,0.286147,2999,cross,Trips-Wordnet-pt,0.318676
0,both,max,average,0.202995,0.133437,2999,cross,Trips-Wordnet-npt,0.168425
0,lookup,max,max,0.328673,0.263367,2999,cross,Trips-Wordnet-npt,0.296367
